In [1]:
import re
import numpy as np
import collections
from collections import Counter
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.tree import DecisionTreeRegressor

from sklearn.tree import DecisionTreeClassifier

In [2]:
samp = pd.read_csv('SampleSubmission (1).csv')
samp.head()

,Id,y
0,0,0.0
1,1,1.0
2,2,0.0
3,3,0.0
4,4,0.0


In [3]:
samp.shape

(37590, 2)

In [32]:
!pip install --upgrade pickle-mixin

Requirement already up-to-date: pickle-mixin in /home/explore/anaconda3/lib/python3.6/site-packages (1.0.2)


In [14]:
!pip install --upgrade pip

Requirement already up-to-date: pip in /home/explore/anaconda3/lib/python3.6/site-packages (19.3.1)


In [18]:
!pip install --upgrade pandas

     |████████████████████████████████| 10.4MB 29.8MB/s eta 0:00:01    |████████████████████████▋       | 8.0MB 29.8MB/s eta 0:00:01
  Found existing installation: pandas 0.23.4
    Uninstalling pandas-0.23.4:
      Successfully uninstalled pandas-0.23.4


In [2]:
import pickle

In [3]:
# with open('train.pkl', 'rb') as f:
#     train = pickle.load(f)
# with open('test.pkl', 'rb') as g:
#     test = pickle.load(g)

train = pd.read_pickle('train.pkl')
test = pd.read_pickle('test.pkl')

In [63]:
train.head()

,Id,c,r,y
0,0,_t_ comment faire pour choisir bonne musique ...,merci,1
1,1,_t_ comment faire pour choisir bonne musique ...,لباس العربي,0
2,2,_t_ comment faire pour choisir bonne musique ...,mkber 9albk,0
3,3,_t_ comment faire pour choisir bonne musique ...,thats kayle taswira mouneka ommha check sword...,0
4,4,_t_ comment faire pour choisir bonne musique ...,صحيح تتعداش أبداا dinar,0


In [64]:
train.shape,test.shape

((85000, 4), (37590, 3))

In [65]:
test.head()

,Id,c,r
0,0,_t_ Yrem _t_ _t_ lords نسيطرو عليهم الغروبات...,Răchëd MhāmDī
1,1,_t_ Yrem _t_ _t_ lords نسيطرو عليهم الغروبات...,انا.منحبش.مسي.انحب.رونالدو
2,2,_t_ Yrem _t_ _t_ lords نسيطرو عليهم الغروبات...,الmarketing like
3,3,_t_ Yrem _t_ _t_ lords نسيطرو عليهم الغروبات...,اخيب حاجة عملتها حياتي شريت اوريدو
4,4,_t_ Yrem _t_ _t_ lords نسيطرو عليهم الغروبات...,troudouha moch


In [4]:
test['y'] = None

In [40]:
df = pd.concat([train, test])

In [41]:
df.head()

,Id,c,r,y,clean,clean_r
0,0,comment faire pour choisir bonne musique po...,merci,1,comment faire pour choisir bonne musique pour ...,merci
1,1,comment faire pour choisir bonne musique po...,لباس العربي,0,comment faire pour choisir bonne musique pour ...,لباس العربي
2,2,comment faire pour choisir bonne musique po...,mkber 9albk,0,comment faire pour choisir bonne musique pour ...,mkber albk
3,3,comment faire pour choisir bonne musique po...,thats kayle taswira mouneka ommha check sword...,0,comment faire pour choisir bonne musique pour ...,thats kayle taswira mouneka ommha check sword ...
4,4,comment faire pour choisir bonne musique po...,صحيح تتعداش أبداا dinar,0,comment faire pour choisir bonne musique pour ...,صحيح تتعداش أبداا dinar


In [42]:
df['c'] = df['c'].apply(lambda x: ' '.join(x.split('_t_')))
df['r'] = df['r'].apply(lambda x: ' '.join(x.split('_t_')))

In [43]:
def translitArabic(text):
    buckwalterMod = {
        'ء': 'c', 'ا': 'A', 'إ': 'A',
        'أ': 'A', 'آ': 'A', 'ب': 'b',
        'ة': 'o', 'ت': 't', 'ث': 'v',
        'ج': 'j', 'ح': 'H', 'خ': 'x',
        'د': 'd', 'ذ': 'V', 'ر': 'r',
        'ز': 'z', 'س': 's', 'ش': 'E',
        'ص': 'S', 'ض': 'D', 'ط': 'T',
        'ظ': 'Z', 'ع': 'C', 'غ': 'g',
        'ف': 'f', 'ق': 'q', 'ك': 'k',
        'ل': 'l', 'م': 'm', 'ن': 'n',
        'ه': 'h', 'ؤ': 'c', 'و': 'w',
        'ى': 'y', 'ئ': 'c', 'ي': 'y',
        }
    for k, v in buckwalterMod.items():
        text = re.sub(k, v, text)
    return(text)

In [44]:
df['clean'] = df['c'].apply(translitArabic)
df['clean_r'] = df['r'].apply(translitArabic)

In [45]:
pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
subs_url = r'url-web'
df['clean'] = df['clean'].replace(to_replace = pattern_url, value = subs_url, regex = True)
df['clean_r'] = df['clean_r'].replace(to_replace = pattern_url, value = subs_url, regex = True)

In [46]:
df['clean'] = df['clean'].str.lower()
df['clean_r'] = df['clean_r'].str.lower()

In [47]:
import string
from string import digits

df['clean'] = df['clean'].apply(lambda x : x.translate(str.maketrans(' ',' ',string.punctuation)))
df['clean'] = df['clean'].apply(lambda x : x.translate(str.maketrans(' ',' ',digits)))
df['clean_r'] = df['clean_r'].apply(lambda x : x.translate(str.maketrans(' ',' ',string.punctuation)))
df['clean_r'] = df['clean_r'].apply(lambda x : x.translate(str.maketrans(' ',' ',digits)))

In [48]:
df['clean'] = df['clean'].str.strip()
df['clean_r'] = df['clean_r'].str.strip()

In [49]:
train = df[pd.notnull(df['y'])]
test = df[pd.isnull(df['y'])]

In [50]:
X1 = train['clean']
X2 = train['clean_r']
y = train['y']

In [51]:
X2.head()

0                                                merci
1                                          lbas alcrby
2                                           mkber albk
3    thats kayle taswira mouneka ommha check sword ...
4                              shyh ttcdae abdaa dinar
Name: clean_r, dtype: object

In [52]:
y.head()

0    1
1    0
2    0
3    0
4    0
Name: y, dtype: object

In [65]:
# Xv = X.apply(lambda x: x.str.strip())
# Xv.head()

In [60]:
Vect = CountVectorizer()

In [61]:
X1_vect1 = Vect.fit_transform(X1)

In [62]:
X_train, X_test, y_train, y_test = train_test_split(X1_vect1, y.astype(int), test_size = 0.2, random_state = 42)

In [63]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((68000, 19931), (17000, 19931), (68000,), (17000,))

In [64]:
Vect2 = CountVectorizer()

In [65]:
X_vec2 = Vect2.fit_transform(X2)

X_train2, X_test2, y_train2, y_test2 = train_test_split(X_vec2, y.astype(int), test_size = 0.2, random_state = 42)

In [66]:
X_train2.shape, X_test2.shape, y_train2.shape, y_test2.shape

((68000, 27880), (17000, 27880), (68000,), (17000,))

In [67]:
df1_vec1 = pd.SparseDataFrame(X_train, columns=Vect.get_feature_names())

/home/explore/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  """Entry point for launching an IPython kernel.
/home/explore/anaconda3/lib/python3.6/site-packages/pandas/core/sparse/frame.py:257: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  sparse_index=BlockIndex(N, blocs, blens),


In [68]:
df2_vec2 = pd.SparseDataFrame(X_train2, columns=Vect2.get_feature_names())

/home/explore/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  """Entry point for launching an IPython kernel.
/home/explore/anaconda3/lib/python3.6/site-packages/pandas/core/sparse/frame.py:257: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  sparse_index=BlockIndex(N, blocs, blens),
/home/explore/anaconda3/lib/python3.6/site-packages/pandas/core/sparse/frame.py:269: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Serie

In [71]:
df1_vec1.shape, df2_vec2.shape, dff1_vec1.shape, dff2_vec2.shape

((68000, 19931), (68000, 27880), (17000, 19931), (17000, 27880))

In [70]:
dff1_vec1 = pd.SparseDataFrame(X_test, columns=Vect.get_feature_names())
dff2_vec2 = pd.SparseDataFrame(X_test2, columns=Vect2.get_feature_names())

/home/explore/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  """Entry point for launching an IPython kernel.
/home/explore/anaconda3/lib/python3.6/site-packages/pandas/core/sparse/frame.py:257: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Series(pd.SparseArray(...))

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  sparse_index=BlockIndex(N, blocs, blens),
/home/explore/anaconda3/lib/python3.6/site-packages/pandas/core/sparse/frame.py:269: FutureWarning: SparseSeries is deprecated and will be removed in a future version.
Use a Series with sparse values instead.

    >>> series = pd.Serie

In [24]:
#Vect = CountVectorizer(ngram_range=(1, 1), stop_words='english', lowercase = True, max_features = 5000)
#Vect = CountVectorizer(stop_words = None, token_pattern = r'(?u)\b\w+\b').fit(X_train) #, token_pattern = r'(?u)\b\w+\b')
#df_train = pd.DataFrame(Vect.transform(X_train).todense(), columns=Vect.get_feature_names())

In [25]:
#df_test= pd.DataFrame(Vect.transform(X_test).todense(), columns=Vect.get_feature_names())

In [56]:
df_train.shape, df_test.shape, y_train.shape, y_test.shape

((68000, 21013), (17000, 20542), (68000,), (17000,))

In [38]:
lm1 = LogisticRegression()

In [40]:
lm1.fit(X_train,y_train)

/home/explore/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [41]:
pred1 = lm1.predict(X_test)

In [42]:
pred1

array([0, 0, 0, ..., 0, 0, 0])

In [43]:
lm1.score(X_test, y_test)

0.8983529411764706

In [44]:
lm2 = LogisticRegression()

In [45]:
lm2.fit(X_train2,y_train2)

/home/explore/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
pred2 = lm2.predict(X_test2)
pred2

array([0, 0, 0, ..., 0, 0, 0])

In [47]:
lm2.score(X_test2, y_test2)

0.8983529411764706

In [72]:
mix = pd.concat([df1_vec1, df2_vec2], axis = 1)

/home/explore/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:392: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  return cls(data, **kwargs)


In [73]:
miy = pd.concat([dff1_vec1, dff2_vec2], axis = 1)

/home/explore/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:392: FutureWarning: SparseDataFrame is deprecated and will be removed in a future version.
Use a regular DataFrame whose columns are SparseArrays instead.

See http://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html#migrating for more.

  return cls(data, **kwargs)


In [74]:
mix.shape, miy.shape, y_train.shape, y_test.shape

((68000, 47811), (17000, 47811), (68000,), (17000,))

In [75]:
lmm = LogisticRegression()

In [80]:
from scipy import sparse

In [89]:
mix.sparse.coo_matrix()

AttributeError: 'SparseFrameAccessor' object has no attribute 'coo_matrix'

In [90]:
import gc

In [91]:
gc.collect()

866

In [92]:
lmm.fit(mix,y_train)

/home/explore/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


MemoryError: 